<a href="https://colab.research.google.com/github/nejukmaster/GroundLIb/blob/main/GroundLib-python/CNN_for_Minecraft_Mat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Generate CNN model for minecraft using .sample file can be extracted with GroundLib Java Plugin in Minecraft

In [ ]:
from tensorflow import keras
from sklearn.model_selection import train_test_split

In [ ]:
model = keras.Sequential()

In [ ]:
conv3D = keras.layers.Conv3D(32,kernel_size=(3,3,4),activation='relu',padding='same', input_shape = (20,20,80,1), kernel_initializer='he_uniform')
pool3D = keras.layers.MaxPooling3D(2)
model.add(conv3D)
model.add(pool3D)

In [ ]:
conv3D = keras.layers.Conv3D(32,kernel_size=(2,2,4),activation='relu',padding='same', kernel_initializer='he_uniform')
pool3D = keras.layers.MaxPooling3D(2)
model.add(conv3D)
model.add(pool3D)

In [ ]:
flatten = keras.layers.Flatten()
dense1 = keras.layers.Dense(200,activation='relu')
dropout1 = keras.layers.Dropout(0.1)
dense2 = keras.layers.Dense(100,activation='relu')
dropout2 = keras.layers.Dropout(0.4)
model.add(flatten)
model.add(dense1)
model.add(dropout1)
model.add(dense2)
model.add(dropout2)

In [ ]:
output = keras.layers.Dense(1,activation='sigmoid')
model.add(output)

In [ ]:
model.summary()

In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

In [ ]:
import glob
 
path_dir = '/gdrive/My Drive/samples/*'
 
file_list = glob.glob(path_dir)
samples = []
samples_path = []
for f in file_list:
  list = glob.glob(f+"/*")
  samples+=list
print(samples)

In [ ]:
#reading test
f = open(samples[0], 'r')
print(f.read())

In [ ]:
arr = []
for s in samples:
  f = open(s,'r')
  arr.append(f.read())
print(len(arr))

In [ ]:
arr2 = []
for s in arr:
  arr2.append(s.split("/"))

In [ ]:
import numpy as np

inputs = np.zeros((200,20,20,80))
targets = np.zeros((200,))
for s in arr2:
  _l = np.zeros((20,20,80))
  for i in range(0,80):
    _s = s[i].split(":")
    del _s[len(_s)-1]
    for j in range(0,20):
      for k in range(0,20):
        _l[k][j][i] = int(s[i].split(":")[k+j*20])
  inputs[arr2.index(s)] = _l

In [ ]:
for i in range(0,len(arr2)):
  targets[i] = int(arr2[i][80])
print(targets)

In [ ]:
from sklearn.model_selection import train_test_split

train_scaled = inputs/255.0
train_scaled = train_scaled.reshape(200,20,20,80,1)
targets = targets.reshape(-1,1)
train_scaled, val_scaled, train_target, val_target = train_test_split(train_scaled, targets, test_size=0.2)

In [ ]:
print(train_scaled.shape)
print(train_target.shape)

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics='accuracy')

checkpoint_cb = keras.callbacks.ModelCheckpoint('best-cnn-model.h5')
early_stopping_cb = keras.callbacks.EarlyStopping(patience=2,
                                                  restore_best_weights=True)

history = model.fit(train_scaled, train_target, epochs=20,
                    validation_data=(val_scaled, val_target),
                    callbacks=[checkpoint_cb, early_stopping_cb])

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['loss'])
plt.xlabel('epoch')
plt.ylabel('loss')
plt.savefig('7_3-01', dpi=300)
plt.show()

In [ ]:
model.evaluate(val_scaled, val_target)

In [ ]:
#test
print(val_scaled[0].shape)
_i = val_scaled[0].reshape(1,20,20,80,1)
print(model.predict(_i))
print(val_target[0])